In [3]:
from collections import defaultdict
import argparse

import numpy as np
import pandas as pd

from collections import Counter
from Bio import SeqIO
import networkx as nx

import matplotlib.pyplot as plt

import sys
import os
sys.setrecursionlimit(500524)

### Functions to contract nodes

### Main Loop
If there are 1000 sequences in the fasta file, the script will randomly choose 100 sequences and repeat the process for 10 times.

If there are fewer than 1000 sequences, comment out the outer loop.

In [25]:
seq_list = []
k_lengths = [500]
fasta = ["../HIV3.fasta"]
outdir = "../HIV3_500mer_100_random/"

outputname = os.path.basename(fasta[0]).split(".")[0]

for seqq in fasta:
    seq_list = seq_list + [(str(a.id),str(a.seq)) for a in list(SeqIO.parse(seqq, "fasta"))]

# random experiments
np.random.shuffle(seq_list)
    
for seqn in range(0,1000,100):
    print("====== Seqn = "+ str(seqn))
    seq_df = pd.DataFrame(seq_list[seqn:seqn+100])
    #seq_df = pd.DataFrame(seq_list).head()
    seq_df.columns = ["name","Sequence"]

    name_dict = list(enumerate(seq_df["name"]))

    for k_length in k_lengths:
        print(" K-mer lengths = "+str(k_length))
        kmer_pos_dict = defaultdict(list)
        kmer_name_dict = defaultdict(list)
        kmer_pos_dict = defaultdict(list)
        kmer_dict = set()
        for i in range(len(seq_df)):
            name = i
#             print(name)
            seq = seq_df.loc[i,"Sequence"]
            for j in range(len(seq)-k_length+1):
                kmer_dict.add(seq[j:j+k_length])
        #     print(i,seq[j:j+k_length])
        kmer_id_list = dict(zip(sorted(list(kmer_dict)), list(range(1,len(kmer_dict)+1))))
        id_kmer_list = dict(zip(list(range(1,len(kmer_dict)+1)),sorted(list(kmer_dict))))

        edges = defaultdict(list)
        for i in range(len(seq_df)):
            name = i
            seq = seq_df.loc[i,"Sequence"]
            edges[0].append(kmer_id_list[seq[:k_length]])
            for j in range(len(seq)-k_length-1):
                edges[kmer_id_list[seq[j:j+k_length]]].append(kmer_id_list[seq[j+1:j+k_length+1]])
                kmer_name_dict[kmer_id_list[seq[j:j+k_length]]].append(name)
                kmer_pos_dict[kmer_id_list[seq[j:j+k_length]]].append(j)
            kmer_name_dict[kmer_id_list[seq[len(seq)-k_length:]]].append(name)
            kmer_pos_dict[kmer_id_list[seq[len(seq)-k_length:]]].append(j)

        G = nx.DiGraph()

        print("Number of nodes: ",len(kmer_id_list))
        num = 0
        edges_list = []
        for j in edges:
            for i in edges[j]:
                G.add_edge(j,i)
                edges_list.append((j,i))
                num+=1
        print("Number of edges: ",num)
        
        print("Contracting nodes...")
        source = [i for i in G.in_degree() if i[1]==0]
        a = DFSOBJ()
        a.DFS(0, G)

        new_G = nx.DiGraph()
        for i in a.sn_adj:
            for j in a.sn_adj[i]:
                new_G.add_edge(i, j)
        print("Number of nodes: " + str(len(new_G.nodes)))
        print("Number of edges: " + str(len(new_G.edges)))
        nx.write_gexf(new_G, outdir+"compactSWIGG_"+outputname+"_"+str(k_length)+"_"+str(seqn)+".gexf")

====== Seqn = 0
=========== K-mer lengths = 500
Number of nodes:  802975
Number of edges:  848167
Contracting nodes...
Number of nodes: 347
Number of edges: 410
====== Seqn = 100
=========== K-mer lengths = 500
Number of nodes:  820466
Number of edges:  849297
Contracting nodes...
Number of nodes: 181
Number of edges: 201
====== Seqn = 200
=========== K-mer lengths = 500
Number of nodes:  820198
Number of edges:  848594
Contracting nodes...
Number of nodes: 248
Number of edges: 285
====== Seqn = 300
=========== K-mer lengths = 500
Number of nodes:  823203
Number of edges:  850823
Contracting nodes...
Number of nodes: 212
Number of edges: 241
====== Seqn = 400
=========== K-mer lengths = 500
Number of nodes:  801298
Number of edges:  849158
Contracting nodes...
Number of nodes: 356
Number of edges: 421
====== Seqn = 500
=========== K-mer lengths = 500
Number of nodes:  818101
Number of edges:  849358
Contracting nodes...
Number of nodes: 212
Number of edges: 238
====== Seqn = 600
======

In [14]:
kmer_appear_times = defaultdict()
pos_appear_times = defaultdict()
for i in kmer_pos_dict:
    times = len(kmer_pos_dict[i])
    kmer_appear_times[i] = times
    for j in kmer_pos_dict[i]:
        pos_appear_times[j] = times

In [793]:
sorted_kmer_appear_times=sorted(kmer_appear_times.keys(), key= lambda x: kmer_appear_times[x])

In [794]:
windowsize=100
kmer_appear_times_list = [[],[]]
i=0
while i < len(kmer_appear_times)-windowsize:
    kmer_appear_times_list[0].append(i)
    times = 0
    for j in range(0,windowsize):
        times += kmer_appear_times[i+j]
    kmer_appear_times_list[1].append(times)
    i = i + windowsize

KeyError: 0